In [1]:
import random

def generate_level(width, height, initial_density=0.3):
    """Generates a random level with a given density of filled cells."""
    level = [['.' for _ in range(width)] for _ in range(height)]
    
    for y in range(height):
        for x in range(width):
            if random.random() < initial_density:
                level[y][x] = '#'  # '#' represents a filled cell
                
    return level

def evaluate_level(level):
    """Evaluation function that counts the number of filled cells."""
    return sum(row.count('#') for row in level)

def mutate_level(level, mutation_rate=0.05):
    """Mutates the level by randomly changing cells."""
    width = len(level[0])
    height = len(level)
    mutated_level = [row[:] for row in level]  # Create a copy

    for y in range(height):
        for x in range(width):
            if random.random() < mutation_rate:
                mutated_level[y][x] = '#' if mutated_level[y][x] == '.' else '.'

    return mutated_level

def reinforce_pcg(width, height, generations=10, mutation_rate=0.05, evaluation_function=evaluate_level):
    """Reinforcement-based PCG that evolves levels using the evaluation function."""
    best_level = generate_level(width, height)
    best_score = evaluation_function(best_level)

    for _ in range(generations):
        mutated_level = mutate_level(best_level, mutation_rate)
        mutated_score = evaluation_function(mutated_level)

        if mutated_score > best_score:  # Maximization
            best_level = mutated_level
            best_score = mutated_score

    return best_level, best_score

def print_level(level):
    """Prints the level to the console."""
    for row in level:
        print(''.join(row))

# Evaluation function for connected regions
def evaluate_connected_regions(level):
    """Evaluation function that encourages connected regions."""
    width = len(level[0])
    height = len(level)
    visited = [[False for _ in range(width)] for _ in range(height)]
    regions = 0

    def dfs(y, x):
        if y < 0 or y >= height or x < 0 or x >= width or visited[y][x] or level[y][x] == '.':
            return
        visited[y][x] = True
        dfs(y + 1, x)
        dfs(y - 1, x)
        dfs(y, x + 1)
        dfs(y, x - 1)

    for y in range(height):
        for x in range(width):
            if level[y][x] == '#' and not visited[y][x]:
                regions += 1
                dfs(y, x)

    return -regions  # Lower number of regions = better connectivity

# Run with original evaluation
width, height, generations = 20, 10, 50
best_level, best_score = reinforce_pcg(width, height, generations)

print("Best Level:")
print_level(best_level)
print(f"Best Score: {best_score}")

# Run with connected regions evaluation
best_level2, best_score2 = reinforce_pcg(width, height, generations, evaluation_function=evaluate_connected_regions)

print("\nBest Level using connected regions evaluation:")
print_level(best_level2)
print(f"Best Score with connected regions: {best_score2}")

Best Level:
###.###..#..##.#..##
.##.#.#.###...#.##..
#....#.##...###..##.
#.#..#.#.###....###.
##.##....##.####..##
##...##.#.#.....#..#
.#.#.####..###..###.
#####.#.#.#.###.#.##
.##.##.##.########.#
##....#.#.####.##..#
Best Score: 113

Best Level using connected regions evaluation:
.###..........#####.
..###..####..##..##.
..###...........##.#
..#######.......####
######.##.......#...
#.###.#.......#...#.
..#..#.....#..#...#.
.....##########...#.
#...........#######.
#..##.......####.#.#
Best Score with connected regions: -8
